In [1]:
# Intentar hacer que no haga falta que quien corra el notebook se descargue antes los datos
# Que se descarguen en la misma ubicación donde esté el notebook
# Leer solo los archivos '*.csv'
# Append de todos los ficheros
# Valorar si leer los archivos horarios o diarios (según tamaño)
# más interesante datos en tiempo real, para ver patrones de comportamiento a lo largo del día, fin de semana...
# pandas, matplotlib
# función para coger un csv, eliminar líneas que no interesen y convertirlo en un df
# usar esa función dentro de otra que haga un append de un número ilimitado de dfs
# El append de todo va a ser imposible
# Dejamos para el final el tema de descargar y unzipear

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('data/Anio201810/abr_mo18.csv', sep=';')
data.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2018,4,1,2.0,V,...,2.0,V,2.0,V,3.0,V,4.0,V,3.0,V
1,28,79,4,1,28079004_1_38,2018,4,2,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
2,28,79,4,1,28079004_1_38,2018,4,3,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
3,28,79,4,1,28079004_1_38,2018,4,4,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
4,28,79,4,1,28079004_1_38,2018,4,5,2.0,V,...,2.0,V,2.0,V,4.0,V,5.0,V,4.0,V


In [4]:
data.shape

(4490, 56)

In [5]:
# data.dtypes

In [6]:
# vamos stackear el df para que las horas sean una columna y las Vs otra
def stack_df(dataframe, columns_remain):
    dataframe = dataframe.set_index(columns_remain)
    dataframe = dataframe.stack().reset_index()
    return dataframe

In [7]:
# mejorable con regexp
v_cols = [col for col in list(data.columns) if col[0] == 'V']

In [8]:
def get_stacked_dataframe(dataframe, cols_remain, cols_to_drop):
    dataframe = dataframe.drop(columns=cols_to_drop)
    dataframe = stack_df(dataframe, cols_remain)
    return dataframe

In [14]:
def add_last_col_to_df(df1, df2):
    df1['new_col'] = df2.iloc[:,-1]
    return df1

In [15]:
def get_final_df(dataframe):
    
    df_h = get_stacked_dataframe(
        dataframe,
        cols_remain=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA'],
        cols_to_drop=[col for col in list(dataframe.columns) if col[0] == 'V']
    )
    
    df_v = get_stacked_dataframe(
        dataframe,
        cols_remain=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA'],
        cols_to_drop=[col for col in list(dataframe.columns) if col[0] == 'H']
    )
    
    final_df = add_last_col_to_df(df_h, df_v)
    
    return final_df

In [16]:
final_dataframe = get_final_df(data)
final_dataframe.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,level_8,0,new_col
0,28,79,4,1,28079004_1_38,2018,4,1,H01,2.0,V
1,28,79,4,1,28079004_1_38,2018,4,1,H02,2.0,V
2,28,79,4,1,28079004_1_38,2018,4,1,H03,2.0,V
3,28,79,4,1,28079004_1_38,2018,4,1,H04,1.0,V
4,28,79,4,1,28079004_1_38,2018,4,1,H05,2.0,V
